In [1]:
## 환경설정

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## 기본 라이브러리

import os
from glob import glob
from pprint import pprint
import json
from pathlib import Path

In [3]:
# 모델 선언

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.9, 
)

In [4]:
## csv 문서 로드
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader("./data/skrc_faq.csv", encoding="utf-8")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

문서의 수: 42
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': './data/skrc_faq.csv', 'row': 0}
--------------------------------------------------
처음 문서의 내용: 
 question: 제주지점에서 이용 가능한 부가용품은 무엇이 있나요?
answer: 고객님의 즐거운 여행을 위해 제주지점만의 다양한 부가용품을 운영하고 있어요.
차량 예약 시 부가상품도 한 번에 예약하고 결제하실 수 있어요. 

• 유아용품 : 유모차, 카시트
• 전기차 여행템  : 카페KIT, 시네마 KIT(국산 전기차 전용, * 니로 제외)
• 여행템(전 차종) :  자전거, 트레킹, 피크닉 세트
• 키즈 굿즈 :  쿨시트, 부스터, 킥보드, 햇빛 가리개


In [5]:
## CSV 문서 메타데이터 변환 및 키워드/요약 추출

from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

# 키워드와 요약을 추출하는 함수
def extract_keyword_and_summary(text, llm):
    """텍스트에서 키워드와 요약을 추출하는 함수"""
    
    # 키워드 추출 프롬프트
    keyword_prompt = f"""
    다음 텍스트에서 핵심 키워드 3-5개를 추출해주세요. 
    쉼표로 구분하여 답변하세요.
    
    텍스트: {text}
    
    키워드:"""
    
    # 요약 추출 프롬프트
    summary_prompt = f"""
    다음 텍스트를 2-3문장으로 요약해주세요.
    
    텍스트: {text}
    
    요약:"""
    
    try:
        # 키워드 추출
        keyword_response = llm.invoke(keyword_prompt)
        keywords = keyword_response.content.strip()
        
        # 요약 추출
        summary_response = llm.invoke(summary_prompt)
        summary = summary_response.content.strip()
        
        return keywords, summary
        
    except Exception as e:
        print(f"키워드/요약 추출 실패: {e}")
        return "키워드 추출 실패", "요약 추출 실패"

# CSV 문서를 새로운 메타데이터 형식으로 변환
def transform_csv_docs_metadata(csv_docs, llm):
    """CSV 문서의 메타데이터를 새로운 형식으로 변환"""
    
    transformed_docs = []
    
    for i, doc in enumerate(csv_docs):
        try:
            # 기존 내용에서 질문과 답변 추출
            content = doc.page_content
            
            # CSV 형식에 따라 질문과 답변 파싱
            if 'question:' in content and 'answer:' in content:
                # 질문과 답변 분리
                parts = content.split('answer:')
                question_part = parts[0].replace('question:', '').strip()
                answer_part = parts[1].strip()
                
                question = question_part
                answer = answer_part
            else:
                # 다른 형식인 경우 전체 내용을 사용
                question = content
                answer = content
            
            # 키워드와 요약 추출
            full_text = f"{question}\n\n{answer}"
            keywords, summary = extract_keyword_and_summary(full_text, llm)
            
            # 새로운 Document 객체 생성
            new_doc = Document(
                page_content=doc.page_content,  # 기존 내용 유지
                metadata={
                    'question_id': i + 1,  # 1부터 시작하는 ID
                    'question': question,
                    'answer': answer,
                    'keyword': keywords,
                    'summary': summary
                }
            )
            
            transformed_docs.append(new_doc)
            
            # 진행상황 출력
            if (i + 1) % 10 == 0:
                print(f"처리 진행률: {i + 1}/{len(csv_docs)}")
                
        except Exception as e:
            print(f"문서 {i+1} 변환 실패: {e}")
            # 실패한 경우 기본값으로 생성
            new_doc = Document(
                page_content=doc.page_content,
                metadata={
                    'question_id': i + 1,
                    'question': '질문 추출 실패',
                    'answer': '답변 추출 실패',
                    'keyword': '키워드 추출 실패',
                    'summary': '요약 추출 실패'
                }
            )
            transformed_docs.append(new_doc)
    
    return transformed_docs

# 메타데이터 변환 실행
print("CSV 문서 메타데이터 변환 시작...")
transformed_docs = transform_csv_docs_metadata(csv_docs, llm)

print(f"\n✅ 변환 완료! 총 {len(transformed_docs)}개 문서")

CSV 문서 메타데이터 변환 시작...
처리 진행률: 10/42
처리 진행률: 20/42
처리 진행률: 30/42
처리 진행률: 40/42

✅ 변환 완료! 총 42개 문서


In [6]:
## 벡터DB에 transformed_docs 저장

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os

# 임베딩 모델 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터DB 저장 경로
vector_db_path = "./vector_db/skrc_faq"

# 기존 벡터DB 삭제
if os.path.exists(vector_db_path):
    import shutil
    shutil.rmtree(vector_db_path)

# 벡터DB 생성 및 저장
vectorstore = Chroma.from_documents(
    documents=transformed_docs,
    embedding=embeddings,
    persist_directory=vector_db_path
)

vectorstore.persist()
print(f"✅ 벡터DB 저장 완료: {len(transformed_docs)}개 문서")

✅ 벡터DB 저장 완료: 42개 문서


C:\Users\SKTelecom\AppData\Local\Temp\ipykernel_26956\2513845666.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [7]:
## Ensemble Retriever를 이용한 검색

from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# 임베딩 모델과 벡터DB 로드
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_db_path = "./vector_db/skrc_faq"

# 벡터 검색기 (Vector Retriever)
vector_retriever = Chroma(
    persist_directory=vector_db_path,
    embedding_function=embeddings
).as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# BM25 검색기 (키워드 기반 검색)
bm25_retriever = BM25Retriever.from_documents(
    documents=transformed_docs
)
bm25_retriever.k = 5

# Ensemble Retriever 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]  # 벡터 검색에 더 높은 가중치
)

print("✅ Ensemble Retriever 생성 완료")

✅ Ensemble Retriever 생성 완료


C:\Users\SKTelecom\AppData\Local\Temp\ipykernel_26956\1136353814.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_retriever = Chroma(


In [8]:
## RAG chain 구성

## RAG Chain - Ensemble Retriever를 이용한 답변 생성

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# RAG를 위한 프롬프트 템플릿 생성
qa_prompt_template = """당신은 별별렌터카의 친절한 고객 상담원입니다. 
아래의 참조 문서를 바탕으로 고객의 질문에 정확하고 친절하게 답변해주세요.

참조 문서:
{context}

고객 질문: {question}

답변 시 다음 사항을 지켜주세요:
1. 참조 문서의 내용을 바탕으로 정확한 정보 제공
2. 친근하고 이해하기 쉬운 언어 사용
3. 필요한 경우 구체적인 예시나 단계별 설명 제공
4. 한국어로 답변

답변:"""

# 프롬프트 템플릿 생성
QA_PROMPT = PromptTemplate(
    template=qa_prompt_template,
    input_variables=["context", "question"]
)

# RAG Chain 생성
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    chain_type_kwargs={
        "prompt": QA_PROMPT,
        "verbose": True
    },
    return_source_documents=True
)

print("✅ RAG Chain 생성 완료")

✅ RAG Chain 생성 완료


In [9]:
## Gradio를 이용한 대화형 별별렌터카 FAQ 챗봇 (에러 수정)

import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 기존 설정들 로드
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_db_path = "./vector_db/skrc_faq"

# 벡터 검색기
vector_retriever = Chroma(
    persist_directory=vector_db_path,
    embedding_function=embeddings
).as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# BM25 검색기
bm25_retriever = BM25Retriever.from_documents(
    documents=transformed_docs
)
bm25_retriever.k = 5

# Ensemble Retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]
)

# LLM 모델
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.9
)

def get_faq_answer(question):
    """FAQ 질문에 대한 답변을 생성하는 함수"""
    
    try:
        # Ensemble Retriever로 관련 문서 검색
        relevant_docs = ensemble_retriever.get_relevant_documents(question, k=3)
        
        # 컨텍스트 생성
        context = ""
        for i, doc in enumerate(relevant_docs):
            context += f"[문서 {i+1}]\n"
            context += f"질문: {doc.metadata['question']}\n"
            context += f"답변: {doc.metadata['answer']}\n"
            context += f"요약: {doc.metadata['summary']}\n\n"
        
        # 수정된 RAG 프롬프트
        prompt = f"""당신은 별별렌터카의 친절한 고객 상담원입니다.

참조 문서:
{context}

고객 질문: {question}

위 참조 문서를 바탕으로 정확하고 친절하게 답변해주세요.

**중요한 규칙:**
1. 명확한 근거가 없으면 "근거없음"으로 답변하세요
2. 답변하기 어려운 질문은 "잘 모르겠습니다"라고 대답하세요
3. 추측이나 일반적인 지식을 이용하지 마세요
4. 참조 문서의 내용만을 바탕으로 답변하세요
5. 답변은 한국어로 작성하고, 필요시 구체적인 예시를 포함하세요

답변:"""

        # 답변 생성
        response = llm.invoke(prompt)
        answer = response.content
        
        # 참조 문서 정보 추가
        reference_info = "\n\n📚 참조 문서:\n"
        for i, doc in enumerate(relevant_docs):
            reference_info += f"• {doc.metadata['question']}\n"
        
        full_answer = answer + reference_info
        
        return full_answer
        
    except Exception as e:
        return f"죄송합니다. 오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 구성
with gr.Blocks(title="별별렌터카 FAQ 챗봇", theme=gr.themes.Soft()) as demo:
    
    # 헤더
    gr.Markdown("# �� 별별렌터카 FAQ 챗봇")
    gr.Markdown("렌터카 이용에 궁금한 점이 있으시면 언제든 물어보세요!")
    
    with gr.Row():
        with gr.Column(scale=3):
            # 챗봇 인터페이스 - 메시지 형식 수정
            chatbot = gr.Chatbot(
                height=500, 
                show_label=False
            )
            
            # 입력 필드와 버튼을 한 줄에 배치
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="질문을 입력하세요...",
                    show_label=False,
                    lines=2,
                    scale=4
                )
                submit_btn = gr.Button("전송", variant="primary", size="lg", scale=1)
            
            # 대화 초기화 버튼
            clear = gr.Button("대화 초기화", variant="secondary")
        
        with gr.Column(scale=1):
            gr.Markdown("### 💡 예시 질문")
            gr.Markdown("""
            • 제주지점에서 이용 가능한 부가용품은 무엇인가요?
            • 렌터카 결제는 어떻게 하나요?
            • 유아용품을 대여할 수 있나요?
            • 전기차 충전료는 어떻게 정산되나요?
            • 예약을 변경하고 싶어요
            """)
    
    def respond(message, chat_history):
        if message.strip() == "":
            return chat_history, ""
        
        bot_message = get_faq_answer(message)
        
        # 메시지 형식을 올바르게 구성
        chat_history.append([message, bot_message])
        return chat_history, ""
    
    # 이벤트 연결
    submit_btn.click(respond, [msg, chatbot], [chatbot, msg])  # 전송 버튼 클릭
    msg.submit(respond, [msg, chatbot], [chatbot, msg])        # Enter 키 입력
    clear.click(lambda: ([], ""), outputs=[chatbot, msg])       # 초기화 버튼

# Gradio 앱 실행
if __name__ == "__main__":
    try:
        # 먼저 7860 포트로 시도
        demo.launch(
            server_name="127.0.0.1",
            server_port=7860,
            share=True,
            show_error=True
        )
    except OSError:
        try:
            # 7860이 사용 중이면 7861로 시도
            print("포트 7860이 사용 중입니다. 포트 7861로 시도합니다...")
            demo.launch(
                server_name="127.0.0.1",
                server_port=7861,
                share=True,
                show_error=True
            )
        except OSError:
            # 자동으로 사용 가능한 포트 찾기
            print("자동으로 사용 가능한 포트를 찾습니다...")
            demo.launch(
                server_name="127.0.0.1",
                server_port=None,  # 자동 포트 할당
                share=True,
                show_error=True
            )

c:\Users\SKTelecom\llm-mvp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\SKTelecom\AppData\Local\Temp\ipykernel_26956\1936241505.py:102: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
